# PROJ.4简介

PROJ.4是开源GIS最著名的地图投影库，它专注于地图投影的表达，以及转换，许多GIS开源软件的投影都直接使用Proj.4的库文件。

介绍它之前，先从最熟悉的ArcGIS的**prj文件**开始，prj文件是一个专门用来**描述坐标系投影**的文件，里面定义了坐标系的名称、长短半轴、基准面、投影参数等一些其他重要的信息（参考[ArcGIS的.prj文件生成proj4格式的字符串](https://blog.csdn.net/wb175208/article/details/54319028)）。当不在arcgis中处理，而需在code中处理坐标问题时，那proj4这一C语言编写的专门用来处理各个坐标系之间的转换的开源库就是最常用的了。它可以实现经纬度坐标转平面坐标或者不同椭球体之间的坐标系转换，运算精度高，在很多GIS软件中都有使用。Proj.4的功能主要有经纬度坐标与地理坐标的转换，坐标系的转换，包括基准变换等。

地图投影的表达方式有多种，可以举个例子，打开网站https://epsg.io/ ，随意搜索一个坐标系，比如Lambert Conformal Conic，选择一个[North America Lambert Conformal Conic](https://epsg.io/102009)，页面往下拉可以看到export选项下有很多格式。这就是地图投影的各种不同类型的表达方式。其中，PROJ.4就是这里用的格式，形式如下所示：

+proj=lcc +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs

可以看到PROJ.4采用一种非常简单明了的投影表达，加号分隔了各个参数项。同时它又具备强大的投影转换功能，因此它是非常吸引人的。而[pyproj](https://github.com/pyproj4/pyproj)是PROJ的一个python接口。使用它也可以很容易进行坐标转换。

使用pip安装很容易：

``` python
pip install pyproj
```

## pyproj简介

参考了[在Python里使用的Proj4](https://www.cnblogs.com/oloroso/p/5749057.html)。

proj 不光是一些应用程序的集合，它更是一个库，其它语言可以来调用它，来进行更高级的开发和应用。在 proj安装上之后，它本身作为库，可以被 C/C++来调用。而 proj本身是一个开源的项目，同时 Python也是一个开源的编程语言。Proj理所应当的能够用在 Python里。在 Python里的 Proj库称为 Pyproj。在 windows和 Linux下都很好安装。如前所述。

Pyporj是 Python下的 proj。可以很方便的对点来进行地图投影转换。同时在它的基础上开发出更高级的应用。Pyproj包里包括两个类，Proj类和Geod类。

Proj 类可以进行地图投影的变换从经纬度转为 xy投影坐标，也可以反转。也可以在不同的地图投影之间转换。

Geod类可以很方便的计算地球上任意两点的大圆距离，以及它们的相对方位。同时，也可根据方位和大圆距离来反算出另一点的经纬度。其处理的输入坐标可以是 python数组，list元组，scalar 或者 numpy/Numeric/numarray arrays。在导入 Pyproj后可以用其内部的函数 test()会运行一些例子。

### Proj类

当初始化一个 Proj 类的实例时，地图投影的参数设置可以用关键字/值的形式。关键字和值的形式也可以用字典或关键字参数，或者一个 **proj4 字符串**（这个就是前面说的proj形式的投影表达）。

如果可选的关键字“errcheck”为真的话（默认为假）， 一个异常将会被给出，如果转换无效的话。 如果为假，且转换无效时，没有异常抛出，会返会一个无效值 1.e30。

可以将经纬度分别存入一个list或array。可以进行更高效率的转换。输入的值应当是双精度（如果输入的不是，它们将会被转为双精度）。虽然Proj可以和numpy and regular python array objects,python sequences and scalars,但是用array对象速度快一些。

首先初始化一个投影：Proj4投影控制参数或者是以字典形式给出，或者是以关键字参数给出，也可以用proj4的形式给出字符串。

In [1]:
from pyproj import Proj
p = Proj(proj='utm',zone=10,ellps='WGS84') # use kwargs
# p = Proj('+proj=utm +zone=10 +ellps=WGS84') # use proj4 string
x,y=p(105,36)
print('%.3f,%.3f' %(x,y))

-3937003.942,14745198.051


上述代码即将经纬度坐标转换为平面坐标。

接下来使用transform()函数进行不同投影的转换。用法：

transform(p1, p2, x, y, z=None, radians=False) x2, y2, z2 = transform(p1, p2, x1, y1, z1, radians=False)

在p1和p2两个投影之间进行投影转换，将把在p1坐标系下的点(x1,y1,z1)转换到p2所定义的投影中去。z1是可选的，如果没有设z1，会假定为0，并仅仅返回x2,y2。

In [11]:
from pyproj import Proj
albers=Proj('+proj=aea +lon_0=105 +lat_1=25 +lat_2=47 +ellps=krass') 
utm=Proj(proj='utm',zone=48,ellps='WGS84') # 499999.99999999773 3983948.4533356656
# utm=Proj(proj='utm',zone=48,ellps='krass')  # 499999.99999999773 3984019.058813517 不同datum计算结果不同
albers_x,albers_y=albers(105,36) 
albers_x,albers_y 

(0.0, 3847866.972516728)

In [12]:
utm_x,utm_y=utm(105,36) 
print(utm_x,utm_y )

499999.99999999773 3983948.4533356656


In [13]:
from pyproj import transform
to_utm_x,to_utm_y = transform(albers,utm,albers_x ,albers_y) 
print(to_utm_x,to_utm_y )

499999.99999999773 3983948.453335665


更多基准面的转换和投影的转换的例子，可以参考[入门-Python-GIS坐标转换](https://zhuanlan.zhihu.com/p/75000381)

场景设置如下：

记录一组当地坐标系的坐标的文本文件（此处暂考虑文本文件，其实只要是有格式说明的或白皮书的GIS格式，都可以采用批处理来完成，只不过添加相应的格式读取类库来进行数据预处理，比如shp，geojson等等，选择文本文件的原因，是本文关注点是坐标转换。），如何将这组坐标叠加到高德地图上？（高德地图其实是web mercator，但按国测局要求进行了偏移，网络上大家称为国测局gcj02).

In [2]:
import os 
from pyproj import CRS
from pyproj import Transformer
from converter import wgs84_to_gcj02    #参见注意事项

input_file = 'input.txt'
output_file = 'output.txt'

#当地坐标系转WGS84
# 定义当地坐标系，格式是wkt格式，所以使用from_wkt
from_crs = CRS.from_wkt('PROJCS["local",GEOGCS["GCS_Xian_1980",DATUM["D_Xian_1980",SPHEROID["Xian_1980",6378140.0,298.257]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",135005.0014],PARAMETER["False_Northing",-1999781.9795],PARAMETER["Central_Meridian",109.75],PARAMETER["Scale_Factor",1.0],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]')
# 要转换的坐标系
to_crs = CRS.from_epsg(4326)
# 定义transformer
transformer = Transformer.from_crs(from_crs, to_crs, always_xy=True)

#WGS84转Web Mercator，下面的定义方式和上面一样，为什么要转两次呢，因为这里面用到的wgs84_to_gcj02加偏移的函数是定义在wgs84下的，得先转过来才能用这个函数
from_crs_2 = CRS.from_epsg(4326)
to_crs_2 = CRS.from_epsg(3857)
transformer_2 = Transformer.from_crs(from_crs_2, to_crs_2, always_xy=True)

with open(output_file, "w") as fo:    
    with open(input_file, "r") as fi:
          while True:
                line = fi.readline()     # 逐行读取
                if not line:
                    break
                else:
                   array = line.split(",")    # x,y 逗号分隔
                   x1,y1 = transformer.transform(array[0], array[1])   # 当地坐标系转WGS84
                   x2,y2 = wgs84_to_gcj02(x1, y1)                      # gcj02 坐标偏移
                   x3,y3 = transformer_2.transform(x2, y2)             # WGS84转Web Mercator
                   fo.write(",".join(["{:.6f}".format(x3),"{:.6f}".format(y3),'\n']))     # 输出到新文件

print('All Done!')

ModuleNotFoundError: No module named 'converter'

注意：

- 首先请参考pyproj4的API
- 此处借用github上[WGS84转GCJ02的Python脚本](https://link.zhihu.com/?target=https%3A//github.com/wandergis/coordTransform_py/blob/master/coordTransform_utils.py)，请自行下载。